# SUMMARY

## [Import libraries and files](#ch0)

# 1 - [Model](#ch1)

## Import libraries and files

In [3]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

# Cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

from heapq import nlargest

In [4]:
data = pd.read_csv('/Users/alexandremonod/Python/OC ML/p2/csv/cleaned.csv')
data.dropna(subset = ["product_name"], inplace = True)

# Add "code" column
raw = pd.read_csv('/Users/alexandremonod/Python/OC ML/p2/csv/products.csv', sep='\t')
raw = raw["code"]
data = pd.merge(data, raw, left_index=True, right_index=True)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,35,36,37,38,39,48) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<a id="ch1"></a>
## 1 - Model

In [5]:
# New column, with Yuka2 score

# Yuka score goes from -15 to 40.
# I want additives and organic_score to have an influence of 7% and 10%
# Then I need nutrition_score extended on 83, organic_score on 10 and additives on 7

data["score_additives"] = np.nan
for i in data["additives_n"]:       
    if i <= 4:
        data["score_additives"] = 0
    if i > 4 and i <= 12 :
        data["score_additives"] = 2
    if 13 < i <20:
        data["score_additives"] = 5
    if i > 20:
        data["score_additives"] = 7


for i in data.flag_organic :
    if i == 1 :
        data.score_bio = -10

data["Yuka2"] = data["nutrition-score-fr_100g"]/55*83 + data["score_additives"] + data.score_bio

In [34]:
def best_similar_product(code_product):
    category_product = data[data.code == code_product].pnns_groups_2.values[0]
    if category_product == "unknown":
        return("Category unknown")
    else:
        data_category = data[data.pnns_groups_2 == category_product]
        liste_category = data_category.product_name.tolist()
        name_product = data.loc[data.code == code_product, 'product_name'].values[0]
        liste_cosinus = []
        for i in liste_category:
            liste_vectoriser = [name_product, i]
            # Initialize an instance of tf-idf Vectorizer
            tfidf_vectorizer = TfidfVectorizer(strip_accents = 'unicode')
            # Generate the tf-idf vectors for the corpus
            tfidf_matrix = tfidf_vectorizer.fit_transform(liste_vectoriser)
            cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
            # Select value of the correlation
            liste_cosinus.append(cosine_sim[1,0])
        
        # Find the index
        max_values = nlargest(5, liste_cosinus)[1:] # Drop first product, same as the one registered
        #max_indexes = []
        #[max_indexes.append(liste_cosinus.index(i)) for i in max_values]
        for i in max_values:
            liste_max_indexes =  [j for j,x in enumerate(liste_cosinus) if x == i]
        print(liste_max_indexes)
        
        # Select best score
        data_category.reset_index(inplace=True)
        data_best = data_category[data_category.index.isin(liste_max_indexes)]
        print(data_best)
        best_product = data_best[data_best.Yuka2 == data_best.Yuka2.min()]   
        return(best_product[[           # Display some informations about the product
            "product_name", "brands"
            ,"additives_n", "nutrition_grade_fr"
            , "energy_100g", "fat_100g"
            , "saturated-fat_100g", "carbohydrates_100g"
            , "sugars_100g", "fiber_100g"
            , "proteins_100g", "salt_100g"
            , "sodium_100g"
             ]])

In [7]:
data.loc[data.pnns_groups_2 != "unknown"]

,product_name,brands,countries_fr,additives_n,nutrition_grade_fr,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,...,proteins_100g,salt_100g,sodium_100g,nutrition-score-fr_100g,pnns_groups_2,labels_fr,flag_organic,code,score_additives,Yuka2
174,Salade Cesar,Kirkland Signature,Canada,0.0,c,NaN,12.0,7.00,23.00,0.0,...,22.000,2.1600,0.850000,6.000,Vegetables,NaN,1,290616,2,1.054545
175,Danoises à la cannelle roulées,Kirkland Signature,Canada,10.0,e,NaN,14.4,9.60,54.10,28.1,...,4.790,0.9220,0.363000,20.875,Biscuits and cakes,NaN,1,394710,2,23.502273
177,Flute,Waitrose,Royaume-Uni,0.0,e,NaN,4.3,0.60,51.30,4.2,...,9.200,1.2000,0.472441,20.000,Bread,NaN,1,1071894,2,22.181818
180,Chaussons tressés aux pommes,Kirkland Signature,Canada,5.0,c,NaN,10.7,2.00,38.70,24.7,...,3.330,0.6470,0.255000,9.000,Biscuits and cakes,NaN,1,1938067,2,5.581818
182,lentilles vertes,Bertrand Lejeune,France,0.0,e,NaN,1.7,0.30,13.40,1.3,...,12.900,0.5800,0.228346,20.500,Legumes,"Bio,Bio européen,AB Agriculture Biologique",0,5200016,2,22.936364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320759,Лимон — лайм,Пеликан,Russie,NaN,d,NaN,0.2,0.04,14.60,13.6,...,0.500,0.0100,0.003937,15.375,Fruits,NaN,1,9901198001583,2,15.202273
320761,Roussette du Bugey (2011),Roussette du Bugey,France,NaN,e,NaN,0.0,0.00,3.05,0.5,...,0.339,0.0100,0.003937,14.750,Alcoholic beverages,AOC,0,9906410000009,2,14.259091
320763,Thé vert Earl grey,Lobodis,France,0.0,c,21.0,0.2,0.20,0.50,0.5,...,0.500,0.0254,0.010000,2.000,Non-sugared beverages,"Commerce équitable,Bio,Bio européen,Max Havela...",0,99111250,2,-4.981818
320765,Rillette d'oie,"Sans marque,D.Lambert",France,0.0,e,NaN,18.0,6.80,1.00,0.7,...,20.000,2.0000,0.787402,18.750,Processed meat,NaN,1,9935010000003,2,20.295455
